In [ ]:
import requests
import bs4
from tqdm import tqdm
import pandas as pd
import json
import re
import numpy as np
from collections import Counter
import plotly.express as px

In [ ]:
df_1= pd.read_csv('Job Applications.csv')
df_2= pd.read_csv('Job Applications_1.csv')
df_3= pd.read_csv('Job Applications_2.csv')
df_4= pd.read_csv('Job Applications_3.csv')
df_5= pd.read_csv('Job Applications_4.csv')

#Read all CSV files and concatenate them into a single DataFrame

final_df= pd.concat([df_1, df_2, df_3, df_4, df_n], ignore_index=True)
final_df['Application Date'] = pd.to_datetime(final_df['Application Date'], format='%m/%d/%y, %I:%M %p')


In [ ]:


# Function to scrape job details from LinkedIn
def scrape_linkedin_job_details(job_url):
    try:
        response = requests.get(job_url)
        if response.status_code != 200:
            return None
        
        soup = bs4.BeautifulSoup(response.text, 'html.parser')
        
        # Extract job title
        title_element = soup.find('h1', class_='top-card-layout__title')
        title = title_element.text.strip() if title_element else 'Unknown'
        
        # Extract company name
        company_element = soup.find('a', class_='topcard__org-name-link')
        company = company_element.text.strip() if company_element else 'Unknown'
        
        # Extract job description
        description_element = soup.find('div', class_='description__text')
        description = description_element.text.strip() if description_element else 'No description available'
        
        # Extract location
        location_element = soup.find('span', class_='topcard__flavor--bullet')
        location = location_element.text.strip() if location_element else 'Unknown'
        
        return {
            'title': title,
            'company': company,
            'description': description,
            'location': location
            }
        
    
    except Exception as e:
        print(f"Error scraping {job_url}: {e}")
        return None

scrape_linkedin_job_details(final_df['Job Url'].iloc[0])  # Example usage with the first job URL

In [ ]:


all_jobs = []
rows = []  # for pandas

for i in tqdm(range(len(final_df))):
    job_url = final_df['Job Url'].iloc[i]   # ⚡ you were still using final_df instead of final_df
    job_details = scrape_linkedin_job_details(job_url)
    
    if job_details:
        details_dict = job_details[1] if isinstance(job_details, list) else job_details
        all_jobs.append({job_url: details_dict})
        
        # flatten for DataFrame
        row = {"link": job_url}
        row.update(details_dict)
        rows.append(row)

# Save JSON
with open("job_descriptions.json", "w", encoding="utf-8") as f:
    json.dump(all_jobs, f, indent=4, ensure_ascii=False)

# Save CSV
df = pd.DataFrame(rows)
df.to_csv("job_descriptions.csv", index=False, encoding="utf-8-sig")

print("Saved JSON and CSV ✅")


#These JSON and CSV files contain the scraped job details from LinkedIn which can be used for further analysis or visualization in the app